# Support CC3 – Résolution numérique des EDP 1D d’évolution

## 1️⃣ Équation de Chaleur

[
\partial_t u - \kappa \partial_{xx} u + \alpha u = f(t,x), \quad x\in[0,L],\ t>0
]

**IC :** (u(x,0) = u_0(x))
**BC :** Dirichlet, Neumann ou périodique

---

### A. Différences finies

**Démarche :**

1. **Discrétisation spatiale** : On divise ([0,L]) en (N) points espacés de (\Delta x = L/N).
2. **Approximation de la dérivée seconde** :
   [
   \partial_{xx} u(x_j) \approx \frac{u_{j+1} - 2 u_j + u_{j-1}}{\Delta x^2}
   ]
3. **Schéma temporel** : choix entre explicite ou implicite. Pour explicite :
   [
   u_j^{n+1} = u_j^n + \Delta t \left[ \kappa \frac{u_{j+1}^n - 2 u_j^n + u_{j-1}^n}{\Delta x^2} - \alpha u_j^n + f_j^n \right]
   ]
4. **Conditions aux limites** : Dirichlet (u_0 = u_N = 0) ou périodique (u_0 = u_N).
5. **Boucle temporelle** : mise à jour (u_j^n \to u_j^{n+1}).

---

### B. Approche Spectrale (périodique)

**Démarche :**

1. **Choix de la base** : fonctions exponentielles (e^{i 2\pi k x/L}) pour périodique.
2. **Transformée de Fourier** :
   [
   \hat{u}*k(t) = \frac{1}{N} \sum*{j=0}^{N-1} u_j(t) e^{-i 2 \pi k j / N}
   ]
3. **Équation par mode** :
   [
   \partial_t \hat{u}_k + (\kappa (2\pi k/L)^2 + \alpha) \hat{u}_k = \hat{f}_k(t)
   ]
4. **Intégration temporelle** : exponentielle ou Duhamel pour (f\neq 0) :
   [
   \hat{u}_k(t+\Delta t) = \hat{u}_k(t) e^{-(\kappa (2\pi k/L)^2 + \alpha)\Delta t} + \int_t^{t+\Delta t} e^{-(\kappa (2\pi k/L)^2 + \alpha)(t+\Delta t - s)} \hat{f}_k(s) ds
   ]
5. **Transformée inverse** : revenir à l’espace physique (u_j(t) = \sum_k \hat{u}_k(t) e^{i 2\pi k x_j / L}).

---

### C. Code Python – Chaleur

```python
import numpy as np
from scipy.fft import rfft, irfft
import matplotlib.pyplot as plt

# Paramètres
L, N, dt, T, kappa, alpha = 1.0, 128, 0.001, 0.1, 0.01, 0.1
x = np.linspace(0, L, N, endpoint=False)
u0 = np.sin(2*np.pi*x)  # condition initiale
u = u0.copy()

# Différences finies - explicite
dx = L/N
for n in range(int(T/dt)):
    u_new = u.copy()
    u_new[1:-1] = u[1:-1] + dt*(kappa*(u[2:] - 2*u[1:-1] + u[:-2])/dx**2 - alpha*u[1:-1])
    u = u_new

plt.plot(x, u, label='Différences finies')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()

# Spectrale - périodique
u = u0.copy()
k = np.fft.rfftfreq(N, d=L/N)
for n in range(int(T/dt)):
    u_hat = rfft(u)
    u_hat *= np.exp(-(kappa*(2*np.pi*k)**2 + alpha)*dt)
    u = irfft(u_hat)

plt.plot(x, u, label='Spectral')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()
```

---

## 2️⃣ Équation de Schrödinger linéaire

[
i \partial_t \psi = -\frac12 \partial_{xx} \psi + V(x)\psi, \quad x\in[0,L]
]

**IC :** (\psi(x,0) = \psi_0(x))
**BC :** périodique

---

### A. Différences finies

**Démarche :**

1. Discrétisation spatiale : (x_j = j \Delta x).
2. Laplacien : (\partial_{xx} \psi_j \approx (\psi_{j+1}-2\psi_j+\psi_{j-1})/\Delta x^2)
3. Schéma Crank-Nicolson implicite :
   [
   \left(I + i \frac{dt}{4} H\right) \psi^{n+1} = \left(I - i \frac{dt}{4} H\right) \psi^n
   ]
   où (H = -\frac12 D_{xx} + V) (matrice creuse).
4. Résolution à chaque pas avec solveur linéaire (ex: `scipy.sparse.linalg.spsolve`).

---

### B. Approche Spectrale

**Démarche :**

1. Décomposition périodique : (\psi(x,t) = \sum_k \hat{\psi}_k(t) e^{i 2\pi k x / L}).
2. Équation par mode : (\partial_t \hat{\psi}_k = -i (k^2/2 \hat{\psi}_k + \hat{V}_k \ast \hat{\psi}_k))
3. Split-step : alterner :

   * propagation libre : (\hat{\psi}_k \mapsto \hat{\psi}_k e^{-i k^2/2 dt})
   * potentiel : (\psi \mapsto \psi e^{-i V dt})
4. Transformée inverse pour revenir à l’espace physique.

---

### C. Code Python – Schrödinger

```python
import numpy as np
from scipy.fft import rfft, irfft
import matplotlib.pyplot as plt

L, N, dt, T = 1.0, 128, 0.001, 0.1
x = np.linspace(0, L, N, endpoint=False)
psi0 = np.exp(-100*(x-0.5)**2)
psi = psi0.copy()
k = np.fft.rfftfreq(N, d=L/N)

V = np.zeros_like(x)
for n in range(int(T/dt)):
    # propagation libre
    psi_hat = rfft(psi)
    psi_hat *= np.exp(-1j*0.5*(2*np.pi*k)**2*dt)
    psi = irfft(psi_hat)
    # potentiel
    psi *= np.exp(-1j*V*dt)

plt.plot(x, np.abs(psi), label='Spectral |Psi|')
plt.plot(x, np.abs(psi0), '--', label='IC')
plt.legend()
plt.show()
```

---

## 3️⃣ Équation des Ondes

[
\partial_t^2 u - \partial_{xx} u = f(t,x), \quad x\in[0,L]
]

**IC :** (u(x,0) = u_0(x), \quad \partial_t u(x,0) = v_0(x))
**BC :** Dirichlet homogène

---

### A. Différences finies

**Démarche :**

1. Discrétisation spatiale (\Delta x = L/N)
2. Schéma Leapfrog second ordre :
   [
   u_j^{n+1} = 2 u_j^n - u_j^{n-1} + (\Delta t / \Delta x)^2 (u_{j+1}^n - 2 u_j^n + u_{j-1}^n)
   ]
3. Initialisation :
   [
   u_j^1 = u_j^0 + \Delta t v_0(x_j) + O(\Delta t^2)
   ]
4. Boucle temporelle pour calculer (u^{n+1}).

---

### B. Approche Spectrale

**Démarche :**

1. Base sinus (Dirichlet) : (\phi_k(x) = \sin(\pi (k+1)x/L))
2. Décomposition : (u(x,t) = \sum_k \hat{u}_k(t) \phi_k(x))
3. Équation par mode : (\partial_t^2 \hat{u}_k + \lambda_k \hat{u}_k = \hat{f}_k(t)) avec (\lambda_k = (\pi (k+1)/L)^2)
4. Cas (f=0) : solution analytique
   [
   \hat{u}_k(t) = \hat{u}_k(0) \cos(\sqrt{\lambda_k} t) + \hat{v}_k(0)/\sqrt{\lambda_k} \sin(\sqrt{\lambda_k} t)
   ]
5. Transformée inverse DST pour revenir à l’espace physique.

---

### C. Code Python – Ondes

```python
import numpy as np
from scipy.fftpack import dst, idst
import matplotlib.pyplot as plt

# Paramètres
L, N, dt, T = 1.0, 128, 0.001, 0.1
x = np.linspace(0, L, N)
u0 = np.sin(np.pi*x)
v0 = np.zeros_like(x)

# Différences finies
u = u0.copy()
u_prev = u - dt*v0
dx = L/N

for n in range(int(T/dt)):
    u_next = u.copy()
    u_next[1:-1] = 2*u[1:-1]-u_prev[1:-1]+(dt/dx)**2*(u[2:]-2*u[1:-1]+u[:-2])
    u_prev = u.copy()
    u = u_next

plt.plot(x, u, label='DF Ondes')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()

# Spectral
u_hat = dst(u0,type=1)
v_hat = dst(v0,type=1)
lam = (np.pi*np.arange(1,N-1)/L)**2
u_hat_prev = u_hat - dt*v_hat

for n in range(int(T/dt)):
    u_hat_next = 2*np.cos(np.sqrt(lam)*dt)*u_hat - u_hat_prev
    u_hat_prev = u_hat.copy()
    u_hat = u_hat_next

u_spec = idst(u_hat,type=1)/(2*(N-1))
plt.plot(x, u_spec, label='Spectral Ondes')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()
```




Parfait. Voici la version détaillée pour les **problèmes de convection-diffusion 1D et d’ondes dissipatives**, avec **différences finies** et **approche spectrale**, pas à pas.

---

# 4️⃣ Problème de Convection-Diffusion

[
\partial_t u + c \partial_x u - \kappa \partial_{xx} u = f(t,x), \quad x \in [0,L],\ t>0
]

**IC :** (u(x,0) = u_0(x))
**BC :** Dirichlet, Neumann ou périodique

---

### A. Différences finies

**Démarche :**

1. **Discrétisation spatiale** : (x_j = j \Delta x), (j=0\ldots N).
2. **Approximation des dérivées** :

   * Diffusion : (\partial_{xx} u \approx (u_{j+1}-2 u_j + u_{j-1})/\Delta x^2)
   * Convection : (\partial_x u \approx (u_j - u_{j-1})/\Delta x) (upwind) ou ((u_{j+1}-u_{j-1})/(2\Delta x)) (central)
3. **Schéma temporel** : explicite (CFL limité) :
   [
   u_j^{n+1} = u_j^n - c \frac{\Delta t}{\Delta x} (u_j^n-u_{j-1}^n) + \kappa \frac{\Delta t}{\Delta x^2} (u_{j+1}^n - 2 u_j^n + u_{j-1}^n) + dt f_j^n
   ]
4. **Boucle temporelle** : mise à jour (u^n \to u^{n+1}).

**Remarque** : CFL condition : (\Delta t \le \min(\Delta x/c, \Delta x^2/(2\kappa))).

---

### B. Approche Spectrale (périodique)

**Démarche :**

1. Décomposition périodique : (u(x,t) = \sum_k \hat{u}_k(t) e^{i 2\pi k x/L})
2. Convection et diffusion dans l’espace spectral :
   [
   \partial_t \hat{u}_k = - i 2\pi k c / L \hat{u}_k - \kappa (2\pi k / L)^2 \hat{u}_k + \hat{f}_k
   ]
3. Intégration temporelle :
   [
   \hat{u}_k(t+\Delta t) = \hat{u}_k(t) e^{-(i 2\pi k c / L + \kappa (2\pi k / L)^2)\Delta t} + \int_t^{t+\Delta t} e^{-(i 2\pi k c / L + \kappa (2\pi k / L)^2)(t+\Delta t-s)} \hat{f}_k(s) ds
   ]
4. Transformée inverse FFT pour obtenir (u(x,t)).

---

### C. Code Python – Convection-Diffusion

```python
import numpy as np
from scipy.fft import rfft, irfft
import matplotlib.pyplot as plt

# Paramètres
L, N, dt, T = 1.0, 128, 0.001, 0.1
c, kappa = 1.0, 0.01
x = np.linspace(0, L, N, endpoint=False)
u0 = np.sin(2*np.pi*x)
u = u0.copy()
dx = L/N

# Différences finies - upwind pour convection
for n in range(int(T/dt)):
    u_new = u.copy()
    u_new[1:] -= c*dt/dx*(u[1:]-u[:-1])
    u_new[1:-1] += kappa*dt/dx**2*(u[2:]-2*u[1:-1]+u[:-2])
    u = u_new

plt.plot(x, u, label='DF Convection-Diffusion')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()

# Spectral - périodique
u = u0.copy()
k = np.fft.rfftfreq(N, d=L/N)
for n in range(int(T/dt)):
    u_hat = rfft(u)
    u_hat *= np.exp(-(1j*2*np.pi*k*c + kappa*(2*np.pi*k)**2)*dt)
    u = irfft(u_hat)

plt.plot(x, u, label='Spectral Convection-Diffusion')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()
```

---

# 5️⃣ Équation d’Ondes Dissipative

[
\partial_t^2 u + \gamma \partial_t u - c^2 \partial_{xx} u = f(t,x), \quad x\in[0,L]
]

**IC :** (u(x,0)=u_0(x),\ \partial_t u(x,0)=v_0(x))
**BC :** Dirichlet homogène

---

### A. Différences finies

**Démarche :**

1. Discrétisation spatiale (\Delta x = L/N).
2. Schéma Leapfrog dissipatif :
   [
   u_j^{n+1} = \frac{(2-\gamma \Delta t) u_j^n - (1-\gamma \Delta t/2) u_j^{n-1} + (c \Delta t / \Delta x)^2 (u_{j+1}^n - 2 u_j^n + u_{j-1}^n) + dt^2 f_j^n}{1+\gamma \Delta t /2}
   ]
3. Initialisation :
   [
   u_j^1 = u_j^0 + dt v_0(x_j) + \frac{dt^2}{2} (c^2 (u_{j+1}^0 - 2 u_j^0 + u_{j-1}^0)/dx^2 - \gamma v_0(x_j) + f_j^0)
   ]
4. Boucle temporelle pour (n=1,2,\dots).

---

### B. Approche Spectrale

**Démarche :**

1. Base sinus (Dirichlet) : (\phi_k(x) = \sin(\pi(k+1)x/L))
2. Décomposition : (u(x,t) = \sum_k \hat{u}_k(t) \phi_k(x))
3. Équation par mode :
   [
   \partial_t^2 \hat{u}_k + \gamma \partial_t \hat{u}_k + \lambda_k \hat{u}_k = \hat{f}_k(t),\quad \lambda_k = (\pi(k+1)/L)^2 c^2
   ]
4. Solution analytique ou schéma semi-implicite (par exemple, Leapfrog modifié pour dissipation).
5. Transformée inverse DST pour retrouver (u(x,t)).

---

### C. Code Python – Ondes Dissipatives

```python
import numpy as np
from scipy.fftpack import dst, idst
import matplotlib.pyplot as plt

# Paramètres
L, N, dt, T = 1.0, 128, 0.001, 0.1
c, gamma = 1.0, 0.1
x = np.linspace(0, L, N)
u0 = np.sin(np.pi*x)
v0 = np.zeros_like(x)

# Différences finies dissipatives
u = u0.copy()
u_prev = u - dt*v0
dx = L/N

for n in range(int(T/dt)):
    u_next = ((2 - gamma*dt)*u - (1 - gamma*dt/2)*u_prev + (c*dt/dx)**2*(np.roll(u,-1)-2*u+np.roll(u,1))) / (1 + gamma*dt/2)
    u_prev = u.copy()
    u = u_next

plt.plot(x, u, label='DF Ondes dissipatives')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()

# Spectral dissipative
u_hat = dst(u0,type=1)
v_hat = dst(v0,type=1)
lam = (np.pi*np.arange(1,N-1)/L)**2 * c**2
u_hat_prev = u_hat - dt*v_hat

for n in range(int(T/dt)):
    u_hat_next = ((2 - gamma*dt)*u_hat - u_hat_prev)/(1 + gamma*dt/2)
    u_hat_prev = u_hat.copy()
    u_hat = u_hat_next

u_spec = idst(u_hat,type=1)/(2*(N-1))
plt.plot(x, u_spec, label='Spectral Ondes dissipatives')
plt.plot(x, u0, '--', label='IC')
plt.legend()
plt.show()
```

